In [42]:
from bs4 import BeautifulSoup
import requests
from lasscraping.scraping import time_random

In [3]:
url = 'https://fcf.com.br/competicoes/'

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 OPR/122.0.0.0'}
html = requests.get(url,headers=headers)

soup = BeautifulSoup(html.text, 'html.parser' )

In [25]:
def request_link(url):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 OPR/122.0.0.0'}
    html = requests.get(url,headers=headers)
    soup = BeautifulSoup(html.text, 'html.parser' )
    return soup

### Extraindo dados matriz

In [33]:
def get_links(id_menu):
    id_menu 
    link_profissionais = soup.find('li', {'id':id_menu}).find_all('li')
    link_profissionais.pop(-1)
    lista = []
    for link in link_profissionais:
        nome = link.get_text()
        print(nome)
        href = link.a['href']
        print(href)
        lista.append([nome,href])
    return lista

In [ ]:
comp_profissionais = get_links('menu-item-75195')
comp_profissionais.pop(0)  # Remover o de 2026
comp_nao_profissionais = get_links('menu-item-84377')

In [112]:
def get_link_table(comp):
    'Pegando link das lista de sumulas do site de acordo com a lista de campeonatos'
    waiting = time_random(0.5,1.4)
    print(f'Time waiting: {waiting}')
    qtd = len(comp)
    for i in range(0,qtd):
        soup_teste = request_link(comp[i][1])
        print('acessando: ', comp[i][1])
        link_tabela = soup_teste.find('a', string='Tabela')['href']
        comp[i].append(link_tabela)

In [44]:
get_link_table(comp_profissionais)

Time waiting: 1.09
acessando:  https://fcf.com.br/competicoes/serie-a-2025/
acessando:  https://fcf.com.br/copa-sc-2025/
acessando:  https://fcf.com.br/competicoes/recopa-2025/
acessando:  https://fcf.com.br/catarinense-sicoob-serie-b-2025/
acessando:  https://fcf.com.br/serie-c-2025/


### Arrumando tabela matriz

In [50]:
import pandas as pd
df_pro = pd.DataFrame(comp_profissionais)
df_pro['Tipo_Competicao'] = 'Profissional'
df_nao_pro = pd.DataFrame(comp_nao_profissionais)
df_nao_pro['Tipo_Competicao'] = 'Não Profissional'

In [51]:
dfs = [df_nao_pro,df_pro]
df = pd.concat(dfs, ignore_index=True)

In [58]:
novos_nomes = {
    0: 'Categoria',
    1: 'URL_FCF',
    2: 'URL_SIGOL',
}
df = df.rename(columns=novos_nomes)

### funcoes pós tabela matriz

In [ ]:
import re
def get_link_sumulas(url_egol):
    '''
    Recebe um link do egol e pega todos links de sumula e armazena em uma lista de link
    '''
    soup = request_link(url_egol)
    sumulas = soup.find_all('title="súmula')
    print(len(sumulas), 'Sumulas encontradas')
    lista_link_sumulas = []
    for sumula in sumulas:
        link_sumulas = sumula.parent.parent
        lista_link_sumulas.append(link_sumulas['href'])
    return lista_link_sumulas

In [114]:
from dataclasses import dataclass, asdict, fields

@dataclass
class Tabela:
    campeonato:str
    fase:str
    jogo:str
    rodada:str
    data:str
    horario:str
    local:str
    ocorrencia_observacoes:str
    observacoes_eventuais:str
    link_sumula:str

def scraping_sumula(lista_link_sumulas):
    'Entra no link da sumula e pega os dados da sumula'
    soup = request_link(lista_link_sumulas)
    campeonato = soup.find('p', string=re.compile('Campeonato')).parent.next_sibling.p.get_text()
    fase = soup.find('p', string=re.compile('Fase')).parent.next_sibling.p.get_text()
    jogo = soup.find('p', string=re.compile('Jogo')).parent.next_sibling.p.get_text()
    rodada = soup.find('p', string=re.compile('Rodada')).parent.next_sibling.p.get_text()
    data = soup.find('p', string=re.compile('Data')).parent.next_sibling.p.get_text()
    horario = soup.find('p', string=re.compile('Horário')).parent.next_sibling.p.get_text()
    local = soup.find('p', string=re.compile('Local')).parent.next_sibling.p.get_text()
    oc_ob = soup.find('font', string=re.compile('OCORRÊNCIAS/OBSERVAÇÕES')).parent.parent.parent.next_sibling.find('font').get_text()
    oc_ev = soup.find('font', string=re.compile('OBSERVAÇÕES EVENTUAIS')).parent.parent.parent.next_sibling.find('font').get_text()
    
    registro = Tabela(
        campeonato=campeonato,
        fase=fase,
        jogo=jogo,
        rodada=rodada,
        data=data,
        horario=horario,
        local=local,
        ocorrencia_observacoes=oc_ob,
        observacoes_eventuais=oc_ev,
        link_sumula=lista_link_sumulas
    )
    return registro


In [122]:
print(f'Esperando: {time_random(1,4)}')

Esperando: 2.16


In [ ]:
registros = []
for df_row in df['URL_SIGOL']:
    print('Acessando url:')
    print(df_row)
    print(f'Esperando: {time_random(1,2)} segundos \n')
    
    lista_url_sumulas = get_link_sumulas(df_row) # Retorna lista de todas as sumulas do link especificado
    
    for url in lista_url_sumulas:
        print(f'Esperando: {time_random(1,2)} segundos \n')
        registro = scraping_sumula(url)
        registros.append(registro)
        print('Registro adicionado!')
        print(registro)
        break

In [ ]:
def processar_url_sigol(url_sigol):
    """Processa uma única URL do SIGOL e raspa todas as súmulas associadas."""
    print(f'Acessando URL: {url_sigol}')
    
    # 1. Espera entre acessos ao SIGOL
    print(f'Tempo aguardado: {time_random(1,2)} segundos \n')
    
    lista_url_sumulas = get_link_sumulas(url_sigol)
    
    registros_locais = []
    
    for url_sumula in lista_url_sumulas:
        # 2. Espera entre acessos à súmula
        print(f'Tempo aguardado: {time_random(1,2)} segundos \n')
        
        # O scraping_sumula é a tarefa principal de I/O
        registro = scraping_sumula(url_sumula) 
        registros_locais.append(registro)
        
        # Note: você tinha um 'break' aqui no seu código original. 
        # Se você o mantiver, ele só processará a primeira súmula de cada URL SIGOL.
        # Vou assumir que você QUER remover o 'break' para processar todas as súmulas:
        # break 
        
    return registros_locais

In [147]:
import concurrent.futures
from tqdm import tqdm # 1. Importe a função tqdm

# ... (Seu código da classe Tabela e funções de scraping)

# Máximo de threads/tarefas simultâneas que você quer rodar.
MAX_THREADS = 5 

registros_finais = []
urls_para_processar = df['URL_SIGOL'].tolist() # Lista de URLs SIGOL
total_urls = len(urls_para_processar)

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    # O executor.map() aplica a função 'processar_url_sigol' a cada item.
    resultados = executor.map(processar_url_sigol, urls_para_processar)
    
    # 2. Envolva o resultado do executor.map() com tqdm()
    # O argumento 'total' é necessário para que o tqdm saiba quantos itens esperar.
    # O 'desc' adiciona um rótulo à barra.
    for lista_registros in tqdm(resultados, total=total_urls, desc="Processando URLs SIGOL"):
        registros_finais.extend(lista_registros)

print(f"\n--- Processamento Concluído ---")
print(f"Total de {len(registros_finais)} registros coletados.")

Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=669&SelStop2=669&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Relat%F3rio+de+execu%E7%E3o
Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=643&SelStop2=643&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Run+Report
Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=644&SelStop2=644&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Run+Report
Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=645&SelStop2=645&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Run+Report
Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=648&SelStop2=648&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport

Processando URLs SIGOL:   0%|          | 0/19 [00:00<?, ?it/s]

Tempo aguardado: 1.44 segundos 

Tempo aguardado: 1.52 segundos 

Tempo aguardado: 1.58 segundos 

Tempo aguardado: 1.92 segundos 

Tempo aguardado: 1.96 segundos 

30 Sumulas
42 Sumulas
96 Sumulas
96 Sumulas
96 Sumulas
Tempo aguardado: 1.53 segundos 

Tempo aguardado: 1.52 segundos 

Tempo aguardado: 1.28 segundos 

Tempo aguardado: 1.51 segundos 

Tempo aguardado: 1.53 segundos 

Tempo aguardado: 1.22 segundos 

Tempo aguardado: 1.53 segundos 

Tempo aguardado: 1.63 segundos 

Tempo aguardado: 1.65 segundos 

Tempo aguardado: 1.8 segundos 

Tempo aguardado: 1.73 segundos 

Tempo aguardado: 1.38 segundos 

Tempo aguardado: 1.81 segundos 

Tempo aguardado: 1.7 segundos 

Tempo aguardado: 1.94 segundos 

Tempo aguardado: 1.6 segundos 

Tempo aguardado: 1.23 segundos 

Tempo aguardado: 1.12 segundos 

Tempo aguardado: 1.26 segundos 

Tempo aguardado: 1.7 segundos 

Tempo aguardado: 1.31 segundos 

Tempo aguardado: 1.79 segundos 

Tempo aguardado: 1.72 segundos 

Tempo aguardado: 1.71 seg

Processando URLs SIGOL:   5%|▌         | 1/19 [01:17<23:14, 77.46s/it]

Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=664&SelStop2=664&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Run+Report
Tempo aguardado: 1.31 segundos 

Tempo aguardado: 1.43 segundos 

Tempo aguardado: 1.97 segundos 

Tempo aguardado: 1.69 segundos 

Tempo aguardado: 1.91 segundos 

Tempo aguardado: 1.38 segundos 

48 Sumulas
Tempo aguardado: 1.61 segundos 

Tempo aguardado: 1.69 segundos 

Tempo aguardado: 1.26 segundos 

Tempo aguardado: 1.72 segundos 

Tempo aguardado: 1.97 segundos 

Tempo aguardado: 1.06 segundos 

Tempo aguardado: 1.27 segundos 

Tempo aguardado: 1.92 segundos 

Tempo aguardado: 1.37 segundos 

Tempo aguardado: 1.28 segundos 

Tempo aguardado: 1.78 segundos 

Tempo aguardado: 1.57 segundos 

Tempo aguardado: 1.81 segundos 

Tempo aguardado: 1.28 segundos 

Tempo aguardado: 1.17 segundos 

Tempo aguardado: 1.94 segundos 

Tempo aguardado: 1.65 segundos 

Tempo aguardado: 1.43 segundos 

...

Acessando URL: https://egol.fcf.com.br/sisgol/DERW700B.asp?SelStart1=2025&SelStop1=2025&SelStart2=683&SelStop2=683&SelStart3=&SelStop3=&SelStart4=&SelStop4=&SelStart5=&SelStop5=&RunReport=Relat%F3rio+de+execu%E7%E3o
Tempo aguardado: 1.73 segundos 

Tempo aguardado: 1.79 segundos 

Tempo aguardado: 1.29 segundos 

Tempo aguardado: 1.12 segundos 

14 Sumulas
Tempo aguardado: 1.29 segundos 

6 Sumulas
Tempo aguardado: 1.28 segundos 

Tempo aguardado: 1.07 segundos 

Tempo aguardado: 1.77 segundos 

Tempo aguardado: 1.74 segundos 

Tempo aguardado: 1.48 segundos 

Tempo aguardado: 1.07 segundos 

Tempo aguardado: 1.12 segundos 

Tempo aguardado: 1.09 segundos 

Tempo aguardado: 1.1 segundos 

Tempo aguardado: 1.27 segundos 

Tempo aguardado: 1.69 segundos 

Tempo aguardado: 1.19 segundos 

Tempo aguardado: 1.75 segundos 

Tempo aguardado: 1.28 segundos 
Tempo aguardado: 1.47 segundos 


Tempo aguardado: 1.14 segundos 

Tempo aguardado: 1.42 segundos 

Tempo aguardado: 1.59 segundos 

Tempo

Processando URLs SIGOL:  42%|████▏     | 8/19 [07:18<08:23, 45.76s/it]

Tempo aguardado: 1.39 segundos 

Tempo aguardado: 1.37 segundos 

Tempo aguardado: 1.64 segundos 

Tempo aguardado: 1.71 segundos 

Tempo aguardado: 1.38 segundos 

Tempo aguardado: 1.87 segundos 

Tempo aguardado: 1.21 segundos 

Tempo aguardado: 1.38 segundos 

Tempo aguardado: 1.33 segundos 

Tempo aguardado: 1.03 segundos 

Tempo aguardado: 1.29 segundos 

Tempo aguardado: 1.52 segundos 

Tempo aguardado: 1.73 segundos 

Tempo aguardado: 1.14 segundos 

Tempo aguardado: 1.97 segundos 

Tempo aguardado: 1.69 segundos 

Tempo aguardado: 2.0 segundos 

Tempo aguardado: 1.16 segundos 

Tempo aguardado: 1.03 segundos 

Tempo aguardado: 1.26 segundos 

Tempo aguardado: 1.52 segundos 

Tempo aguardado: 1.25 segundos 

Tempo aguardado: 1.34 segundos 

Tempo aguardado: 1.82 segundos 

Tempo aguardado: 1.16 segundos 

Tempo aguardado: 1.04 segundos 

Tempo aguardado: 1.41 segundos 

Tempo aguardado: 1.61 segundos 

Tempo aguardado: 1.35 segundos 

Tempo aguardado: 1.98 segundos 

Tempo aguar

Processando URLs SIGOL: 100%|██████████| 19/19 [08:17<00:00, 26.16s/it]


--- Processamento Concluído ---
Total de 829 registros coletados.


In [169]:
df_final = pd.DataFrame(registros_finais)

In [170]:
len(df_final.campeonato.unique())

19

In [ ]:
# Converte a coluna 'data'
df_final['data'] = pd.to_datetime(df_final['data'])

In [172]:
df_final = df_final.query('data > "2025-06-01"')

In [176]:
df_final.sort_values(by='data', inplace=True)

In [179]:
import os
caminho = os.path.join('data', 'fcf_dados.csv')

df_final.to_csv(caminho, index=False)

In [182]:
df_final

,campeonato,fase,jogo,rodada,data,horario,local,ocorrencia_observacoes,observacoes_eventuais,link_sumula
778,CAMPEONATO CATARINENSE SÉRIE B 2025,TURNO,NAÇÃO x CAMBORIÚ,1ª RODADA,2025-06-02,20:00,Arena Joinville / Joinville,Foi respeitado um minuto de silêncio em homena...,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
777,CAMPEONATO CATARINENSE SÉRIE B 2025,TURNO,CARLOS RENAUX x FLUMINENSE,1ª RODADA,2025-06-02,19:30,Augusto Bauer / Brusque,Nada consta,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
265,CAMPEONATO CATARINENSE SUB-20 SÉRIE A 2025,INICIAL - TURNO/RETURNO,CONCÓRDIA x CHAPECOENSE,9ª RODADA,2025-06-03,19:00,Domingos Machado de Lima / Concórdia,Nada consta,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
267,CAMPEONATO CATARINENSE SUB-20 SÉRIE A 2025,INICIAL - TURNO/RETURNO,BARRA x BRUSQUE,10º RODADA,2025-06-04,15:00,Novo CT do Barra / Itajaí,Informo que o médico da partida foi a Dra. MAR...,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
180,CAMPEONATO CATARINENSE SUB-17 SÉRIE A 2025,INICIAL - TURNO/RETURNO,CHAPECOENSE x CRICIÚMA,11ª RODADA,2025-06-04,16:00,Arena Condá / Chapecó,"Aos 25 minutos do 2º tempo, o jogo foi paralis...",Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
...,...,...,...,...,...,...,...,...,...,...
637,CAMPEONATO ESTADUAL NÃO PROFISSIONAL ADULTO 2025,FINAL,JOAO PESSOA x ALIANÇA,FINAL,2025-11-15,15:00,João Lúcio da Costa / Jaraguá do Sul,Nada consta,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
772,COPA SANTA CATARINA 2025,FINAL,JOINVILLE x FIGUEIRENSE SAF,,2025-11-16,16:00,ARENA JOINVILLE / Joinville,RELATO 1: Aos 23 minutos do primeiro tempo foi...,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
574,COPA SC SUB-18 2025,FINAL,MARCÍLIO DIAS x NAÇÃO,,2025-11-20,19:00,Dr. Hercílio Luz / Itajaí,Nada consta,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
540,COPA SC SUB-16 2025,FINAL,FIGUEIRENSE SAF x CRICIÚMA,,2025-11-22,15:00,CFT Cambirela / Palhoça,Médica da Partida- DRA. BRUNA SARTORI/CRM 3818...,Nada consta,http://egol.fcf.com.br/SISGOL/WDER0700_Sumula....
